In [1]:
from h5io_browser.base import _open_hdf, _read_hdf
import h5py
import posixpath

In [2]:
import numpy as np

In [3]:
def list_groups_and_nodes(hdf, h5_path):
    """
    Get the list of groups and list of nodes from an open HDF5 file

    Args:
        hdf (h5py.File): file handle of an open HDF5 file
        h5_path (str): path inside the HDF5 file

    Returns:
        list, list: list of groups and list of nodes
    """
    groups = set()
    nodes = set()
    try:
        h = hdf[h5_path]
        for k in h.keys():
            if isinstance(h[k], h5py.Group):
                groups.add(k)
            else:
                nodes.add(k)
    except KeyError:
        pass
    return list(groups), list(nodes)

In [4]:
def get_h5_path(h5_path, name):
    """
    Combine the current h5_path with the relative path

    Args:
        h5_path (str): absolute path of the node in the hdf5 file
        name (str): relative path to be added to the absolute path

    Returns:
        str: combined path
    """
    return posixpath.join(h5_path, name)

In [5]:
def read_dict_from_hdf(
    file_name, h5_path, recursive=False, group_paths=[], slash="ignore"
):
    """
    Read data from HDF5 file into a dictionary - by default only the nodes are converted to dictionaries, additional
    sub groups can be specified using the group_paths parameter.
    Args:
       file_name (str): Name of the file on disk
       h5_path (str): Path to a group in the HDF5 file from where the data is read
       recursive (bool): Load all subgroups recursively
       group_paths (list): list of additional groups to be included in the dictionary, for example:
                           ["input", "output", "output/generic"]
                           These groups are defined relative to the h5_path.
       slash (str): 'ignore' | 'replace' Whether to replace the string {FWDSLASH} with the value /. This does
                    not apply to the top level name (title). If 'ignore', nothing will be replaced.
    Returns:
       dict:     The loaded data. Can be of any type supported by ``write_hdf5``.
    """

    def get_dict_from_nodes(store, h5_path, slash="ignore"):
        """
        Load all nodes from an HDF5 path into a dictionary
        Args:
            store (str): Name of the file on disk, or file-like object.  Note: for files created with the 'core'
                         driver, HDF5 still requires this be non-empty.:
            h5_path (str): Path to a group in the HDF5 file from where the data is read
            slash (str): 'ignore' | 'replace' Whether to replace the string {FWDSLASH} with the value /. This does
                         not apply to the top level name (title). If 'ignore', nothing will be replaced.
        Returns:
            dict:        The loaded data. Can be of any type supported by ``write_hdf5``.
        """
        return {
            n: _read_hdf(
                hdf_filehandle=store,
                h5_path=get_h5_path(h5_path=h5_path, name=n),
                slash=slash,
            )
            for n in list_groups_and_nodes(hdf=store, h5_path=h5_path)[1]
        }

    def resolve_nested_dict(group_path, data_dict):
        """
        Turns a dict with a key containing slashes into a nested dict.  {'/a/b/c': 1} -> {'a': {'b': {'c': 1}
        Args:
            group_path (str): path inside the HDF5 file the data_dictionary was loaded from
            data_dict (dict): dictionary with data loaded from the HDF5 file
        Returns:
            dict: hierarchical dictionary
        """
        groups = group_path.split("/")
        nested_dict = data_dict
        for g in groups[::-1]:
            nested_dict = {g: nested_dict}
        return nested_dict

    def get_groups_hdf(hdf, h5_path):
        """
        Get all sub-groups of a given HDF5 path
        Args:
            hdf (str): Name of the file on disk, or file-like object.  Note: for files created with the 'core'
                       driver, HDF5 still requires this be non-empty.:
            h5_path (str): Path to a group in the HDF5 file from where the data is read
        Returns:
            list: list of HDF5 groups
        """
        try:
            h = hdf[h5_path]
            group_lst = []
            for group in [h[k].name for k in h.keys() if isinstance(h[k], h5py.Group)]:
                group_lst += [group] + get_groups_hdf(hdf=hdf, h5_path=group)
            return group_lst
        except KeyError:
            return []

    def merge_dict(main_dict, add_dict):
        """
        Merge two dictionaries recursively

        Args:
            main_dict (dict): The primary dictionary, the secondary dictionary is merged into
            add_dict (dict): The secondary dictionary which is merged in the primary dictionary

        Returns:
            dict: The merged dictionary with all keys
        """
        for k, v in add_dict.items():
            if k in main_dict.keys() and isinstance(v, dict):
                main_dict[k] = merge_dict(main_dict=main_dict[k], add_dict=v)
            else:
                main_dict[k] = v
        return main_dict

    if recursive and len(group_paths) > 0:
        raise ValueError(
            "Loading subgroups can either be defined by the group paths ",
            group_paths,
            " or by the recursive ",
            recursive,
            " parameter. Specifying both lead to this ValueError.",
        )

    with _open_hdf(file_name, mode="r") as store:
        output_dict = get_dict_from_nodes(store=store, h5_path=h5_path, slash=slash)
        if h5_path == "/" and recursive:
            group_paths = [g[1:] for g in get_groups_hdf(hdf=store, h5_path=h5_path)]
        elif h5_path[0] != "/" and recursive:
            group_paths = [
                g[len("/" + h5_path) + 1 :]
                for g in get_groups_hdf(hdf=store, h5_path="/" + h5_path)
            ]
        elif recursive:
            group_paths = [
                g[len(h5_path) + 1 :]
                for g in get_groups_hdf(hdf=store, h5_path=h5_path)
            ]
        for group_path in group_paths:
            output_dict = merge_dict(
                main_dict=output_dict,
                add_dict=resolve_nested_dict(
                    group_path=group_path,
                    data_dict=get_dict_from_nodes(
                        store=store,
                        h5_path=get_h5_path(h5_path=h5_path, name=group_path),
                        slash=slash,
                    ),
                ),
            )
    return output_dict

In [6]:
from pyiron_atomistics import Project

In [7]:
pr = Project("test")

In [8]:
pr.remove_jobs(recursive=True, silently=True)

  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
structure = pr.create.structure.ase.bulk("Al", cubic=True)

In [10]:
len(structure)

4

In [11]:
job = pr.create.job.Sphinx("sx")

In [12]:
job.structure = structure

In [13]:
job.calc_minimize()

In [14]:
job.run()

The job sx was saved and received the ID: 1


In [15]:
job_dict = read_dict_from_hdf(
    file_name=job.project_hdf5.file_name,
    h5_path="/",
    recursive=True,
    group_paths=[],
    slash='ignore',
)
job_dict

{'sx': {'job_id': 1,
  'VERSION': '2.6.1',
  'TYPE': "<class 'pyiron_atomistics.sphinx.sphinx.Sphinx'>",
  'server': {'NAME': 'Server',
   'TYPE': "<class 'pyiron_base.jobs.job.extension.server.generic.Server'>",
   'OBJECT': 'Server',
   'DICT_VERSION': '0.1.0',
   'user': 'pyiron',
   'host': 'jupyter-pyiron-2ddev-2dpyiron-2dhdf5-2dformat-2dpfz859ts',
   'run_mode': 'modal',
   'queue': None,
   'qid': None,
   'cores': 1,
   'threads': 1,
   'new_h5': True,
   'structure_id': None,
   'run_time': None,
   'memory_limit': None,
   'accept_crash': False},
  'HDF_VERSION': '0.1.0',
  'DICT_VERSION': '0.1.0',
  'status': 'finished',
  'executable': {'NAME': 'Executable',
   'TYPE': "<class 'pyiron_base.jobs.job.extension.executable.Executable'>",
   'OBJECT': 'Executable',
   'DICT_VERSION': '0.1.0',
   'executable': {'NAME': 'DataContainer',
    'TYPE': "<class 'pyiron_base.storage.datacontainer.DataContainer'>",
    'OBJECT': 'DataContainer',
    'HDF_VERSION': '0.2.0',
    'VERSION':

In [16]:
file_name = job.project_hdf5.file_name
file_name

'/home/jovyan/test/sx.h5'

In [17]:
def print_attrs(name, obj):
    print(name)
    for key, val in obj.attrs.items():
        print("    %s: %s" % (key, val))

In [18]:
with h5py.File(file_name, 'r') as f:
    f.visititems(print_attrs)

sx
sx/DICT_VERSION
    TITLE: unicode
sx/HDF_VERSION
    TITLE: unicode
sx/NAME
    TITLE: unicode
sx/OBJECT
    TITLE: unicode
sx/TYPE
    TITLE: unicode
sx/VERSION
    TITLE: unicode
sx/executable
    TITLE: json
sx/input
sx/input/generic
sx/input/generic/DICT_VERSION
    TITLE: unicode
sx/input/generic/NAME
    TITLE: unicode
sx/input/generic/OBJECT
    TITLE: unicode
sx/input/generic/TYPE
    TITLE: unicode
sx/input/generic/VERSION
    TITLE: unicode
sx/input/generic/data_dict
    TITLE: json
sx/input/generic_dict
    TITLE: json
sx/input/interactive
    TITLE: json
sx/input/parameters
sx/input/parameters/CheckOverlap__index_18
    TITLE: int
sx/input/parameters/Ediff__index_10
    TITLE: float
sx/input/parameters/EmptyStates__index_4
    TITLE: int
sx/input/parameters/EnCut__index_1
    TITLE: int
sx/input/parameters/Estep__index_9
    TITLE: int
sx/input/parameters/HDF_VERSION
    TITLE: unicode
sx/input/parameters/Istep__index_21
    TITLE: int
sx/input/parameters/KJxc__index_12